# 1.1 Birth Rates

The data on US births, provided by the CDC is in `data/births.csv`.

Reproduce the following plot of births by gender over time given the data:

![](births_gender.png)

Note the `1e6` on the y axis for scale

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv('data/births.csv')

grouped = df.groupby(['year', 'gender'], as_index=False).sum()
males = grouped[grouped['gender'] == 'M']
females = grouped[grouped['gender'] == 'F']

ax = plt.subplot(1,1,1)
ax.plot(males.year, males.births, label="Male")
ax.plot(females.year, females.births, label="Female")
ax.legend(title="gender")
plt.xlabel('year')
plt.ylabel('total births per year')
plt.yscale("linear")
plt.ticklabel_format(axis="y", style="sci", scilimits=(0,0))

/home/jlmy/anaconda3/lib/python3.7/site-packages/matplotlib/cbook/__init__.py:2064: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  x[:, None]
/home/jlmy/anaconda3/lib/python3.7/site-packages/matplotlib/axes/_base.py:248: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  x = x[:, np.newaxis]
/home/jlmy/anaconda3/lib/python3.7/site-packages/matplotlib/axes/_base.py:250: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  y = y[:, np.newaxis]


# 1.2 Births anomalies

This was analyzed by beloved statistician Andrew Gelman [here](http://andrewgelman.com/2012/06/14/cool-ass-signal-processing-using-gaussian-processes/), leading to this plot:

![](births_gp100.png)

Explain all three plots in Gelman's figure. 

**1.2:** What is the periodic component? What is the residual? Use your research skills to learn then explain it (in english).

# 1.3 Holiday Anomalies Plot

Reproduce *as best you can* the first of the 3 figures from Andrew Gelman's blog post (your plot may have small differences)

**1.3.1:** Reproduce the births line in a plot. Hint: Make the x axis a `pd.datetime` object

**1.3.2:** Reproduce the `smoothed` line. Hint: use a rolling window average

**1.3.3:** Reproduce the entire figure with the mean line as a horizontal. You can make the y axis total births instead of a % deviation from mean axis (they'll look the same anyway)

In [2]:
# 1.3.1

fr = df.copy()
fr['day'] = fr['day'].fillna(1)
fr['day'] = fr['day'].astype(int)
fr['month'] = fr['month'].astype(int)
fr['year'] = fr['year'].astype(int)


# fr['date'] = pd.to_datetime(fr[['year','month', 'day']])
grouped_day = fr.groupby(['year','month', 'day'], as_index=False).sum()
grouped_day['date'] = pd.to_datetime(dict(year=grouped_day.year, month=grouped_day.month, day=grouped_day.day))
# print(grouped_day)
# print(df)
print(grouped_day)


# ax = plt.subplot(1,1,1)
# ax.plot(males.year, males.births, label="Male")
# ax.plot(females.year, females.births, label="Female")
# ax.legend(title="gender")

ValueError: cannot assemble the datetimes: unconverted data remains: 9

# 2. Recipe Database

### 2.1 

Load the JSON recipe database we saw in lecture 4.

How many of the recipes are for breakfast food? Hint: The `description` would contain the work "breakfast"

### 2.2 A simple recipe recommender

Let's build a recipe recommender: given a list of basic ingredients, find a recipe that uses all those ingredients.

Here is the list of ingredients that can be asked for:

```
['salt', 'pepper', 'oregano', 'sage', 'parsley',
 'rosemary', 'tarragon', 'thyme', 'paprika', 'cumin']
```

**Hint:** Build a new column for each of the ingredients that indicates whether that ingredient is in the recipe.

**example:**
```
recommend_ingredients(["parsley", "paprika", "tarragon"], df)

result: 
# The rows where these 3 ingredients are in the recipe
[2069, 74964, 93768, 113926, 137686, 140530, 158475, 158486, 163175, 165243]
```

# 3. Movies!

Recall the [Movies Dataset](https://www.kaggle.com/rounakbanik/the-movies-dataset) from lecture 4. It's made up of several tables which we've played with in lecture 4.

The tables have common columns (`id` and `movie_id`) around which you can merge and join tables.

### 3.1 best director

Your task is to find **The best director** in terms of average ratings of his movies. This can be from the `ratings` or `ratings_small` table, or simply the vote average in the `metadata` table. The director can be found in the `cast` table.

You will have to use all of your skills to get this done, between using groupbys and merging multiple tables together

In [85]:
"""
From class:
"""
import gzip
import json
movies_url = {
"movies_metadata": "1RLvh6rhzYiDDjPaudDgyS9LmqjbKH-wh",
"keywords": "1YLOIxb-EPC_7QpkmRqkq9E6j7iqmoEh3",
"ratings": "1_5HNurSOMnU0JIcXBJ5mv1NaXCx9oCVG",
"credits": "1bX9othXfLu5NZbVZtIPGV5Hbn8b5URPf",
"ratings_small": "1fCWT69efrj4Oxdm8ZNoTeSahCOy6_u6w",
"links_small": "1fh6pS7XuNgnZk2J3EmYk_9jO_Au_6C15",
"links": "1hWUSMo_GwkfmhehKqs8Rs6mWIauklkbP",
}

def read_gdrive(url):
    """
    Reads file from Google Drive sharing link
    """
    path = 'https://drive.google.com/uc?export=download&id='+url
    return pd.read_csv(path)

def read_zip(url):
    with gzip.open('https://drive.google.com/uc?export=download&id='+url, 'r') as f:
        # Extract each line
        data = (line.strip().decode() for line in f)
        # Reformat so each line is the element of a list
        data_json = f"[{','.join(data)}]"
        # read the result as a JSON
    return pd.read_json(data_json)

mf = read_gdrive(movies_url["movies_metadata"])
"""
File size was throw a reCaptacha on GDrive, couldn't fetch, so had to upload the file in the notebook.
Must have the file in order for this to work.
"""
ratings = read_gdrive(movies_url["ratings_small"])
mc = pd.read_csv('data/credits.csv')
ratings

/home/jlmy/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2901: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
...,...,...,...,...
99999,671,6268,2.5,1065579370
100000,671,6269,4.0,1065149201
100001,671,6365,4.0,1070940363
100002,671,6385,2.5,1070979663


In [86]:

test = pd.DataFrame({
    'id': mc.id
})
def remove_x(row):
    v = set([d['name'] if d['job'] == 'Director' else 'x' for d in row])
    v.discard('x')
    return list(v)
test['director'] = (mc.crew
   .apply(eval)
   .apply(remove_x)
)
"""
If you're not using pandas >= 0.25.1 this won't work
"""
print(pd.__version__)
test = test.explode('director')


1.1.5


In [87]:
"""
Let's assume a film needs to have had at least 10 reviews to count

We'll inspire ourselves on on IMDB's movie rating formulae in order to filter out films:

https://help.imdb.com/article/imdb/track-movies-tv/ratings-faq/G67Y87TFYYP6TWAV#

weighted rating (WR) = (v ÷ (v+m)) × R + (m ÷ (v+m)) × C

Where:

R = average for the movie (mean) = (Rating)
v = number of votes for the movie = (votes)
m = minimum votes required to be listed in the Top 250 (currently 25,000)
C = the mean vote across the whole report

here, let's assume a cut-off of 10 ratings, so, a director needs to have a movie rated 10 times for it to be considered.

with these filtered out films, let's apply a simple score rating equal to:
score (for director) = (average rating of all films / 5 ) * number of unique ratings for all films

This way, we can reward directors for both high ratings, and for having many ratings.

"""

dirs = pd.DataFrame({
    'directors': test.director.unique()
})

    
def pop_dirs(director): 
    ids = test[test['director'] == director]['id'].unique()
    if len(ids) < 8: return 0.0
    check = pd.DataFrame({
        'ids': ids
    })
    ratings_c = ratings[ratings['movieId'].isin(ids)]
    def filter_movie(row):
        filtered = ratings_c[ratings_c['movieId'] == row]
        rating = filtered['rating']
        return pd.Series({
            'rating': rating.mean(),
            'count': float(len(rating))
        })
    check = check.merge(check.ids.apply(filter_movie), left_index=True, right_index=True)
    if 'count' in check:
        filtered2 = check[check['count'] >= 10.]
        num = (filtered2['rating'].mean() / 5)
        denum = filtered2['count'].sum()
        return num / denum
    else:
        return 0.0

dirs['score'] = dirs.directors.apply(pop_dirs)
dirs

,directors,score
0,John Lasseter,0.011894
1,Joe Johnston,0.019429
2,Howard Deutch,0.006226
3,Forest Whitaker,0.000000
4,Charles Shyer,0.000000
...,...,...
19736,Ravi Udyawar,0.000000
19737,Shanra J. Kehl,0.000000
19738,Aaron Osborne,0.000000
19739,Hamid Nematollah,0.000000


In [83]:
dirs.nlargest(35, 'score')

,directors,score
715,Max Ophüls,0.086000
3237,Lloyd Bacon,0.074380
4597,Miklós Jancsó,0.074380
1509,Julio Médem,0.074000
1301,Ralph Bakshi,0.073000
465,Andrew Fleming,0.066116
564,Roger Vadim,0.066116
3751,Al Adamson,0.066000
444,Philip Kaufman,0.065680
974,Joseph H. Lewis,0.065278
